In [1]:
import pandas as pd
df = pd.read_csv('tweets.csv', encoding = 'ISO-8859-1')
df

,textID,text,sentiment
0,cb774db0d1,"I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,negative
2,088c60f138,my boss is bullying me...,negative
3,9642c003ef,what interview! leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...",negative
...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband lo...,negative
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,negative
27478,f67aae2310,Yay good for both of you. Enjoy the break - yo...,positive
27479,ed167662a5,But it was worth it ****.,positive


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   textID     27481 non-null  object
 1   text       27480 non-null  object
 2   sentiment  27481 non-null  object
dtypes: object(3)
memory usage: 644.2+ KB


In [3]:
df.isna().sum()

textID       0
text         1
sentiment    0
dtype: int64

In [4]:
df['text'] = df['text'].fillna('')

In [5]:
df['sentiment'].value_counts()

sentiment
neutral     11118
positive     8582
negative     7781
Name: count, dtype: int64

In [6]:
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': -1,'neutral': 0})

In [7]:
from nltk.corpus import stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [8]:
from nltk.stem.porter import PorterStemmer
port_stem = PorterStemmer()

In [9]:
import re
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower().split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [10]:
df['stemmed_content'] = df['text'].apply(stemming)

In [11]:
X = df['stemmed_content'].values
Y = df['sentiment'].values

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify = Y, random_state = 2)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer 
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [14]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, Y_train)
Y_pred_knn = knn.predict(X_test)

In [15]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, cohen_kappa_score
print('Confusion Matrix:\n', confusion_matrix(Y_test, Y_pred_knn))
print('Classification Report:\n', classification_report(Y_test, Y_pred_knn))
print('Accuracy Score:', accuracy_score(Y_test, Y_pred_knn))
print('Kappa Score:', cohen_kappa_score(Y_test, Y_pred_knn))

Confusion Matrix:
 [[ 148 1400    8]
 [  50 2118   56]
 [  19 1411  287]]
Classification Report:
               precision    recall  f1-score   support

          -1       0.68      0.10      0.17      1556
           0       0.43      0.95      0.59      2224
           1       0.82      0.17      0.28      1717

    accuracy                           0.46      5497
   macro avg       0.64      0.40      0.35      5497
weighted avg       0.62      0.46      0.37      5497

Accuracy Score: 0.46443514644351463
Kappa Score: 0.11637855581182954


In [16]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(iterations=100, 
                           depth=6,          
                           learning_rate=0.1, 
                           loss_function='MultiClass', 
                           random_state=42)

model.fit(X_train, Y_train, cat_features=[])
Y_pred_cat = model.predict(X_test)

0:	learn: 1.0771087	total: 574ms	remaining: 56.8s
1:	learn: 1.0603593	total: 959ms	remaining: 47s
2:	learn: 1.0455266	total: 1.33s	remaining: 43.1s
3:	learn: 1.0331364	total: 1.71s	remaining: 41s
4:	learn: 1.0214163	total: 2.1s	remaining: 40s
5:	learn: 1.0124564	total: 2.49s	remaining: 39s
6:	learn: 1.0052464	total: 2.87s	remaining: 38.1s
7:	learn: 0.9979643	total: 3.27s	remaining: 37.6s
8:	learn: 0.9908185	total: 3.67s	remaining: 37.2s
9:	learn: 0.9844081	total: 4.13s	remaining: 37.2s
10:	learn: 0.9783211	total: 4.58s	remaining: 37.1s
11:	learn: 0.9728791	total: 5.04s	remaining: 36.9s
12:	learn: 0.9684238	total: 5.47s	remaining: 36.6s
13:	learn: 0.9636586	total: 5.91s	remaining: 36.3s
14:	learn: 0.9585455	total: 6.39s	remaining: 36.2s
15:	learn: 0.9547767	total: 6.8s	remaining: 35.7s
16:	learn: 0.9515949	total: 7.22s	remaining: 35.3s
17:	learn: 0.9476489	total: 7.65s	remaining: 34.9s
18:	learn: 0.9441105	total: 8.09s	remaining: 34.5s
19:	learn: 0.9406199	total: 8.55s	remaining: 34.2s


In [17]:
print('Confusion Matrix:\n', confusion_matrix(Y_test, Y_pred_cat))
print('Classification Report:\n', classification_report(Y_test, Y_pred_cat))
print('Accuracy Score:', accuracy_score(Y_test, Y_pred_cat))
print('Kappa Score:', cohen_kappa_score(Y_test, Y_pred_cat))

Confusion Matrix:
 [[ 688  782   86]
 [ 126 1882  216]
 [  39  578 1100]]
Classification Report:
               precision    recall  f1-score   support

          -1       0.81      0.44      0.57      1556
           0       0.58      0.85      0.69      2224
           1       0.78      0.64      0.71      1717

    accuracy                           0.67      5497
   macro avg       0.72      0.64      0.66      5497
weighted avg       0.71      0.67      0.66      5497

Accuracy Score: 0.6676368928506458
Kappa Score: 0.47888837756201275


In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

base_learner = DecisionTreeClassifier(max_depth=1) 
ada = AdaBoostClassifier(estimator=base_learner, n_estimators=50, random_state=40)
ada.fit(X_train, Y_train)
Y_pred_ada = ada.predict(X_test)

In [19]:
print('Confusion Matrix:\n', confusion_matrix(Y_test, Y_pred_ada))
print('Classification Report:\n', classification_report(Y_test, Y_pred_ada))
print('Accuracy Score:', accuracy_score(Y_test, Y_pred_ada))
print('Kappa Score:', cohen_kappa_score(Y_test, Y_pred_ada))

Confusion Matrix:
 [[ 691  748  117]
 [ 126 1816  282]
 [  38  446 1233]]
Classification Report:
               precision    recall  f1-score   support

          -1       0.81      0.44      0.57      1556
           0       0.60      0.82      0.69      2224
           1       0.76      0.72      0.74      1717

    accuracy                           0.68      5497
   macro avg       0.72      0.66      0.67      5497
weighted avg       0.71      0.68      0.67      5497

Accuracy Score: 0.680371111515372
Kappa Score: 0.5019026835400997


In [20]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

param_dist = {'n_estimators': [50, 100, 200],  
              'learning_rate': uniform(0.01, 1.0),  
              'base_estimator__max_depth': randint(1, 4) }

ada = AdaBoostClassifier(estimator=DecisionTreeClassifier())

random_search = RandomizedSearchCV( ada,
                                    param_distributions=param_dist,
                                    n_iter=30,  
                                    cv=3,        
                                    n_jobs=-1,   
                                    random_state=40 )

random_search.fit(X_train, Y_train)

print("Best parameters found: ", random_search.best_params_)
print("Best score found: ", random_search.best_score_)

C:\Users\USER\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:905: FutureWarning: Parameter 'base_estimator' of AdaBoostClassifier is deprecated in favor of 'estimator'. See AdaBoostClassifier's docstring for more details.
  clone(base_estimator).set_params(**self.best_params_)


Best parameters found:  {'base_estimator__max_depth': 3, 'learning_rate': 0.6469222423714195, 'n_estimators': 50}
Best score found:  0.6803584425036391


In [21]:
best_model = random_search.best_estimator_
best_model.fit(X_train, Y_train)
Y_pred = best_model.predict(X_test)

In [22]:
print('Confusion Matrix:\n', confusion_matrix(Y_test, Y_pred_ada))
print('Classification Report:\n', classification_report(Y_test, Y_pred_ada))
print('Accuracy Score:', accuracy_score(Y_test, Y_pred_ada))
print('Kappa Score:', cohen_kappa_score(Y_test, Y_pred_ada))

Confusion Matrix:
 [[ 691  748  117]
 [ 126 1816  282]
 [  38  446 1233]]
Classification Report:
               precision    recall  f1-score   support

          -1       0.81      0.44      0.57      1556
           0       0.60      0.82      0.69      2224
           1       0.76      0.72      0.74      1717

    accuracy                           0.68      5497
   macro avg       0.72      0.66      0.67      5497
weighted avg       0.71      0.68      0.67      5497

Accuracy Score: 0.680371111515372
Kappa Score: 0.5019026835400997


In [23]:
import pickle
with open('model.pkl', 'wb') as file:
    pickle.dump(best_model, file)